<pre>
1. Download all the data in this folder https://drive.google.com/open?id=1Z4TyI7FcFVEx8qdl4jO9qxvxaqLSqoEu. it contains two file both images and labels. The label file list the images and their categories in the following format:
            <b>path/to/the/image.tif,category</b>
            
    where the categories are numbered 0 to 15, in the following order:

    <b>0 letter
    1 form
    2 email
    3 handwritten
    4 advertisement
    5 scientific report
    6 scientific publication
    7 specification
    8 file folder
    9 news article
    10 budget
    11 invoice
    12 presentation
    13 questionnaire
    14 resume
    15 memo</b>
    
2. On this image data, you have to train 3 types of models as given below. You have to split the data into Train and Validation data.

3. Try not to load all the images into memory, use the gernarators that we have given the reference notebooks to load the batch of images only during the train data.
or you can use this method also
<a href='https://medium.com/@vijayabhaskar96/tutorial-on-keras-imagedatagenerator-with-flow-from-dataframe-8bd5776e45c1'>https://medium.com/@vijayabhaskar96/tutorial-on-keras-imagedatagenerator-with-flow-from-dataframe-8bd5776e45c1</a>

<a href='https://medium.com/@vijayabhaskar96/tutorial-on-keras-flow-from-dataframe-1fd4493d237c'>https://medium.com/@vijayabhaskar96/tutorial-on-keras-flow-from-dataframe-1fd4493d237c</a>


4. You are free to choose Learning rate, optimizer, loss function, image augmentation, any hyperparameters. but you have to use the same architechture what we are asking below. 

5. Use tensorboard for every model and analyse your gradients. (you need to upload the screenshots for each model for evaluation)

Note: fit_genarator() method will have problems with the tensorboard histograms, try to debug it, if you could not do use histgrams=0 i.e don't include histograms, check the documentation of tensorboard for more information. 

6. You can check about Transfer Learning in this link - <a href='https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html'>https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html</a>
</pre>

In [3]:
import pandas as pd
import numpy as np
import sys
import os
import random
from pathlib import Path

# Image processing
import imageio
import cv2
import skimage.transform
#from skimage.transform import rescale, resize, downscale_local_mean

# Charts
import matplotlib.pyplot as plt
import seaborn as sns

# ML, statistics
import scipy
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix, roc_curve, auc, roc_auc_score

# Tensorflow
#from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import pandas as pd

In [4]:
all_df = pd.read_csv("/content/drive/MyDrive/Copy of labels_final.csv",dtype=str)
all_df.tail()

,path,label
47995,imagesk/k/q/l/kql82f00/tob07414.87.tif,10
47996,imagesi/i/r/r/irr80c00/2084343690_3692.tif,12
47997,imagesa/a/z/h/azh32d00/2063887153_7176.tif,6
47998,imagesg/g/p/d/gpd45f00/0060075263.tif,8
47999,imagesr/r/o/l/rol45d00/2064701657.tif,1


In [5]:
!unrar x "/content/drive/MyDrive/Copy of rvl-cdip.rar"

Streaming output truncated to the last 5000 lines.
Creating    data_final/imagesy/y/w/v                                  OK
Creating    data_final/imagesy/y/w/v/ywv07e00                         OK
Extracting  data_final/imagesy/y/w/v/ywv07e00/2031300697.tif              95%  OK 
Creating    data_final/imagesy/y/w/v/ywv14c00                         OK
Extracting  data_final/imagesy/y/w/v/ywv14c00/2080388930b.tif             95%  OK 
Creating    data_final/imagesy/y/w/v/ywv15a00                         OK
Extracting  data_final/imagesy/y/w/v/ywv15a00/528410291+-0291.tif         95%  OK 
Creating    data_final/imagesy/y/w/v/ywv90a00                         OK
Extracting  data_final/imagesy/y/w/v/ywv90a00/0060014591.tif              95%  OK 
Creating    data_final/imagesy/y/w/v/ywv90c00                         OK
Extracting  data_final/imagesy/y/w/v/ywv90c00/13527205.tif                95%  OK 
Creating    data_final/imagesy/y/w/v/ywv96c00      

In [ ]:
#from keras.utils import to_categorical
one_hot_label = keras.utils.to_categorical(all_df['label'])

In [ ]:
df=all_df.copy()

In [ ]:
df['label']=one_hot_label

In [ ]:
all_df.head(50)

,path,label
0,imagesv/v/o/h/voh71d00/509132755+-2755.tif,3
1,imagesl/l/x/t/lxt19d00/502213303.tif,3
2,imagesx/x/e/d/xed05a00/2075325674.tif,2
3,imageso/o/j/b/ojb60d00/517511301+-1301.tif,3
4,imagesq/q/z/k/qzk17e00/2031320195.tif,7
5,imagesw/w/u/d/wud95a00/500116895+-6898.tif,4
6,imagesg/g/v/x/gvx25c00/2505142251_2252.tif,1
7,imagese/e/r/m/erm31f00/0001442842.tif,7
8,imagesp/p/e/v/pev40e00/03371370.tif,13
9,imagesi/i/w/i/iwi42d00/2020355073.tif,5


In [6]:
all_df['label'].value_counts()

0     3016
13    3007
12    3006
14    3006
3     3005
8     3003
9     3002
10    3002
7     3000
5     2999
15    2996
4     2994
1     2994
2     2993
11    2992
6     2985
Name: label, dtype: int64

In [7]:
train_df, test_df = train_test_split(all_df, random_state=24)

In [8]:
train_df['label'] = train_df['label'].astype(str)
test_df['label']=test_df['label'].astype(str)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
train_df['label'].value_counts()

3     2294
9     2281
6     2281
1     2278
14    2265
0     2261
5     2251
10    2249
11    2248
4     2243
15    2242
7     2241
2     2234
8     2233
13    2213
12    2186
Name: label, dtype: int64

In [10]:
 # Base train/validation generator
batch_size=32
img_size=(128,128)
datagen = ImageDataGenerator(
            rescale=1./255.,
            validation_split=0.25,
            featurewise_center=False,
            featurewise_std_normalization=False,
            rotation_range=90,
            width_shift_range=0.2,
            height_shift_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
            )
# Train generator
train_generator =datagen.flow_from_dataframe(dataframe=train_df,directory="/content/data_final",x_col="path",
            y_col="label",
            has_ext=True,
            subset="training",
            batch_size=batch_size,
            seed=42,
            shuffle=True,
            class_mode="categorical",
            target_size=img_size)
print('Train generator created')
# Validation generator
val_generator =datagen.flow_from_dataframe(
            dataframe=test_df,
            directory="/content/data_final",
            x_col="path",
            y_col="label",
            has_ext=True,
            subset="validation",
            batch_size=batch_size,
            seed=42,
            shuffle=True,
            class_mode="categorical",
            target_size=img_size)    
print('Validation generator created')

--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.7/logging/__init__.py", line 1025, in emit
    msg = self.format(record)
  File "/usr/lib/python3.7/logging/__init__.py", line 869, in format
    return fmt.format(record)
  File "/usr/lib/python3.7/logging/__init__.py", line 608, in format
    record.message = record.getMessage()
  File "/usr/lib/python3.7/logging/__init__.py", line 369, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/usr/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.7/dist-packages/traitlets/config/application.py", line 845, in launch_instance
    app.start()
  File "/usr/local/lib/pytho

Found 27000 validated image filenames belonging to 16 classes.
Train generator created
Found 3000 validated image filenames belonging to 16 classes.
Validation generator created


--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.7/logging/__init__.py", line 1025, in emit
    msg = self.format(record)
  File "/usr/lib/python3.7/logging/__init__.py", line 869, in format
    return fmt.format(record)
  File "/usr/lib/python3.7/logging/__init__.py", line 608, in format
    record.message = record.getMessage()
  File "/usr/lib/python3.7/logging/__init__.py", line 369, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/usr/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.7/dist-packages/traitlets/config/application.py", line 845, in launch_instance
    app.start()
  File "/usr/local/lib/pytho

### Model-1

<pre>
1. Use <a href='https://www.tensorflow.org/api_docs/python/tf/keras/applications/VGG16'>VGG-16</a> pretrained network without Fully Connected layers and initilize all the weights with Imagenet trained weights. 
2. After VGG-16 network without FC layers, add a new Conv block ( 1 Conv layer and 1 Maxpooling ), 2 FC layers and a output layer to classify 16 classes. You are free to choose any hyperparameters/parameters of conv block, FC layers, output layer. 
3. Final architecture will be <b>INPUT --> VGG-16 without Top layers(FC) --> Conv Layer --> Maxpool Layer --> 2 FC layers --> Output Layer</b>
4. Train only new Conv block, FC layers, output layer. Don't train the VGG-16 network. 

</pre>

In [11]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input, Flatten, Dense,Conv2D,Conv1D
from keras.models import Model
import numpy as np

In [89]:
from tensorflow.keras import layers

In [61]:


#Get back the convolutional part of a VGG network trained on ImageNet
model_vgg16_conv = VGG16(weights='imagenet', include_top=False,input_tensor=Input(shape=(128, 128, 3)))
model_vgg16_conv.summary() 

#Create your own input format (here 3x200x200)
input = Input(shape=(128,128,3),name = 'image_input')

#Use the generated model 
output_vgg16_conv = model_vgg16_conv(input)

#Add the fully-connected layers 
#x = Flatten(name='flatten')(output_vgg16_conv)
x=  Conv2D(16, (3,3), padding="same", activation="relu")(output_vgg16_conv)
x=  MaxPooling2D(pool_size=(2, 2), strides=None, padding="same")(x)
x = Dense(4096, activation='relu', name='fc1')(x)
x = Dense(4096, activation='relu', name='fc2')(x)
x = Flatten(name='flatten')(x)
x = Dense(16, activation='softmax', name='predictions')(x)

#Create your own model 
my_model = Model(inputs=input, outputs=x) 
#for layer in my_model.layers[:18]:
#    layer.trainable = False

#In the summary, weights and layers from VGG part will be hidden, but they will be fit during the training
my_model.summary()


Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_35 (InputLayer)        [(None, 128, 128, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0     

In [62]:
my_model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=['accuracy'])

In [ ]:
my_model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=5,
        validation_data=val_generator,
        validation_steps=800 // batch_size)     
#my_model.save_weights('first_try.h5')

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Model-2

<pre>
1. Use <a href='https://www.tensorflow.org/api_docs/python/tf/keras/applications/VGG16'>VGG-16</a> pretrained network without Fully Connected layers and initilize all the weights with Imagenet trained weights.
2. After VGG-16 network without FC layers, don't use FC layers, use conv layers only as Fully connected layer. any FC layer can be converted to a CONV layer. This conversion will reduce the No of Trainable parameters in FC layers. For example, an FC layer with K=4096 that is looking at some input volume of size 7×7×512 can be equivalently expressed as a CONV layer with F=7,P=0,S=1,K=4096. In other words, we are setting the filter size to be exactly the size of the input volume, and hence the output will simply be 1×1×4096 since only a single depth column “fits” across the input volume, giving identical result as the initial FC layer. You can refer <a href='http://cs231n.github.io/convolutional-networks/#convert'>this</a> link to better understanding of using Conv layer in place of fully connected layers.
3. Final architecture will be VGG-16 without FC layers(without top), 2 Conv layers identical to FC layers, 1 output layer for 16 class classification. <b>INPUT --> VGG-16 without Top layers(FC) --> 2 Conv Layers identical to FC --> Output Layer</b>
3. Train only last 2 Conv layers identical to FC layers, 1 output layer. Don't train the VGG-16 network. 
</pre>

Converting FC layers to CONV layers
It is worth noting that the only difference between FC and CONV layers is that the neurons in the CONV layer are connected only to a local region in the input, and that many of the neurons in a CONV volume share parameters. However, the neurons in both layers still compute dot products, so their functional form is identical. Therefore, it turns out that it’s possible to convert between FC and CONV layers:

In [92]:


#Get back the convolutional part of a VGG network trained on ImageNet
model_vgg16_conv = VGG16(weights='imagenet', include_top=False,input_tensor=Input(shape=(128, 128, 3)))
model_vgg16_conv.summary()

#Create your own input format (here 3x200x200)
input = Input(shape=(128,128,3),name = 'image_input')

#Use the generated model 
output_vgg16_conv = model_vgg16_conv(input)

#Add the fully-connected layers 
#x = Flatten(name='flatten')(output_vgg16_conv)
#x=  Conv2D(16, (3,3), padding="same", activation="relu")(output_vgg16_conv)
#x=  MaxPooling2D(pool_size=(2, 2), strides=None, padding="same")(x)
x = Conv2D(4096, kernel_size=[4,4],strides=(1,1),activation='relu', name='fc1conv2d')(output_vgg16_conv) 
x = Conv2D(4096,kernel_size=[1,1],strides=(1,1),activation='relu', name='fc2conv2d')(x) 
#x = Flatten(name='flatten')(x)
x = Dense(16, activation='softmax', name='predictions')(x)
#x = Conv2D(16, kernel_size=[4,4],activation='softmax', name='predictions')(x)
x = Flatten(name='flatten')(x)
#Create your own model 
my_model2 = Model(inputs=input, outputs=x) 
for layer in my_model2.layers[:18]:
    layer.trainable = False
#In the summary, weights and layers from VGG part will be hidden, but they will be fit during the training
my_model2.summary()


Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_52 (InputLayer)        [(None, 128, 128, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0     

In [93]:
my_model2.compile(optimizer='adam', loss="categorical_crossentropy", metrics=['accuracy'])

In [94]:
my_model2.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=5,
        validation_data=val_generator,
        validation_steps=800 // batch_size)    
#my_model2.save_weights('first_try2.h5')

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
 6/62 [=>............................] - ETA: 3:20 - loss: 2.8282 - accuracy: 0.0705

KeyboardInterrupt: ignored

### Model-3

<pre>
1. Use same network as Model-2 '<b>INPUT --> VGG-16 without Top layers(FC) --> 2 Conv Layers identical to FC --> Output Layer</b>' and train only Last 6 Layers of VGG-16 network, 2 Conv layers identical to FC layers, 1 output layer.
</pre>

In [83]:


#Get back the convolutional part of a VGG network trained on ImageNet
model_vgg16_conv = VGG16(weights='imagenet', include_top=False,input_tensor=Input(shape=(128, 128, 3)))
model_vgg16_conv.summary()

#Create your own input format (here 3x200x200)
input = Input(shape=(128,128,3),name = 'image_input')

#Use the generated model 
output_vgg16_conv = model_vgg16_conv(input)

#Add the fully-connected layers 
#x = Flatten(name='flatten')(output_vgg16_conv)
#x=  Conv2D(16, (3,3), padding="same", activation="relu")(output_vgg16_conv)
#x=  MaxPooling2D(pool_size=(2, 2), strides=None, padding="same")(x)
x = Conv2D(4096, kernel_size=[4,4],activation='relu', name='fc1conv2d')(output_vgg16_conv)
x = Conv2D(4096,kernel_size=[1,1], activation='relu', name='fc2conv2d')(x)
#x = Flatten(name='flatten')(x)
x = Dense(16, activation='softmax', name='predictions')(x)
x = Flatten(name='flatten')(x)
#Create your own model 
my_model3 = Model(inputs=input, outputs=x) 
for layer in my_model3.layers[:12]:
    layer.trainable = False
#In the summary, weights and layers from VGG part will be hidden, but they will be fit during the training
my_model3.summary()


Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_48 (InputLayer)        [(None, 128, 128, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0     

In [86]:
my_model3.compile(optimizer='adam', loss="categorical_crossentropy", metrics=['accuracy'])

In [87]:
my_model3.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=5,
        validation_data=val_generator,
        validation_steps=800 // batch_size)   
#my_model3.save_weights('first_try3.h5')

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
62/62 [==============================] - ETA: 0s - loss: 2.8074 - accuracy: 0.0640

KeyboardInterrupt: ignored